In [2]:
#try 2
import os
import shutil
import pandas as pd

metadata_path = '/users/PZS0694/krishsanghvi/.cache/kagglehub/competitions/animal-clef-2025/metadata.csv'
metadata_df = pd.read_csv(metadata_path)
metadata_df['path'] = metadata_df['path'].str.strip()

# Map from relative path to identity
path_to_identity = dict(zip(metadata_df['path'], metadata_df['identity']))

#/users/PZS0694/krishsanghvi/.cache/kagglehub/competitions/animal-clef-2025
species_paths = {
    'lynx':       '/users/PZS0694/krishsanghvi/.cache/kagglehub/competitions/animal-clef-2025/images/LynxID2025/database',
    'salamander': '/users/PZS0694/krishsanghvi/.cache/kagglehub/competitions/animal-clef-2025/images/SalamanderID2025/database/images',
    'turtle':     '/users/PZS0694/krishsanghvi/.cache/kagglehub/competitions/animal-clef-2025/images/SeaTurtleID2022/database/turtles-data/data/images'
}

new_dataset_path = 'training_data'
os.makedirs(new_dataset_path, exist_ok=True)

image_records = []

for species, src_path in species_paths.items():
    dst_path = os.path.join(new_dataset_path, species)
    os.makedirs(dst_path, exist_ok=True)

    if species == 'turtle':
        for subdir in os.listdir(src_path):
            full_subdir = os.path.join(src_path, subdir)
            if os.path.isdir(full_subdir):
                for img in os.listdir(full_subdir):
                    if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                        rel_path = f"{subdir}/{img}"
                        identity_key = f"images/SeaTurtleID2022/database/turtles-data/data/images/{rel_path}"
                        identity = path_to_identity.get(identity_key)
                        if pd.isna(identity):
                            continue  # Skip images without identity
                        src_img_path = os.path.join(full_subdir, img)
                        dst_img_name = f"{subdir}_{img}"
                        dst_img_path = os.path.join(dst_path, dst_img_name)
                        shutil.copy(src_img_path, dst_img_path)
                        image_records.append([dst_img_path, identity])
    elif species == 'salamander':
        for img in os.listdir(src_path):
            if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                rel_path = f"images/SalamanderID2025/database/images/{img}"
                identity = path_to_identity.get(rel_path)
                if pd.isna(identity):
                    continue
                src_img_path = os.path.join(src_path, img)
                dst_img_path = os.path.join(dst_path, img)
                shutil.copy(src_img_path, dst_img_path)
                image_records.append([dst_img_path, identity])

    else:
        for img in os.listdir(src_path):
            if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                rel_path = f"images/LynxID2025/database/{img}"
                identity = path_to_identity.get(rel_path)

                if pd.isna(identity):
                    continue  # Skip images without identity
                src_img_path = os.path.join(src_path, img)
                dst_img_path = os.path.join(dst_path, img)
                shutil.copy(src_img_path, dst_img_path)
                image_records.append([dst_img_path, identity])

# Create DataFrame and save
df = pd.DataFrame(image_records, columns=['image_path', 'identity'])
df.to_csv('training_data/image_identity.csv', index=False)

# Print image counts
for species in os.listdir(new_dataset_path):
    species_path = os.path.join(new_dataset_path, species)
    if not os.path.isdir(species_path):
        continue
    count = len([f for f in os.listdir(species_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(f"{species}: {count} images")

print("\n✅ Saved identity metadata to training_data/image_identity.csv")


salamander: 1388 images
lynx: 2957 images
turtle: 2780 images

✅ Saved identity metadata to training_data/image_identity.csv


In [3]:
#try 2
import os
import random
import pandas as pd
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

# === Configs ===
target_count = 30000
base_path = 'training_data'
output_csv = os.path.join(base_path, 'full_image_identity.csv')

# === Transform setup ===
augment_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
to_pil = transforms.ToPILImage()

# === Load existing identity CSV created from your first script ===
df_existing = pd.read_csv(os.path.join(base_path, 'image_identity.csv'))
df_existing['image_path'] = df_existing['image_path'].str.strip()

# Map: image filename → identity
filename_to_identity = {
    os.path.basename(row['image_path']): row['identity']
    for _, row in df_existing.iterrows()
}

# Start record list with originals
final_records = df_existing.values.tolist()

# === Augmentation loop ===
for species in os.listdir(base_path):
    species_path = os.path.join(base_path, species)
    if not os.path.isdir(species_path):
        continue

    image_files = [f for f in os.listdir(species_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    image_files = [f for f in image_files if filename_to_identity.get(f) is not None]
    current_count = len(image_files)

    print(f"{species}: {current_count} valid original images found.")

    if current_count >= target_count:
        print(f"✅ {species} already has enough images.")
        continue

    needed = target_count - current_count
    print(f"🔄 Augmenting {needed} images for {species}...")

    for i in tqdm(range(needed)):
        original = random.choice(image_files)
        original_path = os.path.join(species_path, original)
        identity = filename_to_identity[original]

        try:
            base_img = Image.open(original_path).convert('RGB')
            aug_img = augment_transform(base_img)
            aug_img_pil = to_pil(aug_img)

            aug_filename = f"aug_{i}_{original}"
            aug_path = os.path.join(species_path, aug_filename)
            aug_img_pil.save(aug_path)

            final_records.append([aug_path, identity])
        except Exception as e:
            print(f"⚠️ Skipped {original} due to error: {e}")

# === Save final [image_path, identity] CSV ===
df_final = pd.DataFrame(final_records, columns=['image_path', 'identity'])
df_final.to_csv(output_csv, index=False)

print(f"\n✅ Saved full dataset with originals and augmentations to {output_csv}")


salamander: 1388 valid original images found.
🔄 Augmenting 28612 images for salamander...


100%|██████████| 28612/28612 [28:37<00:00, 16.66it/s]


lynx: 2957 valid original images found.
🔄 Augmenting 27043 images for lynx...


100%|██████████| 27043/27043 [13:57<00:00, 32.27it/s]


turtle: 8729 valid original images found.
🔄 Augmenting 21271 images for turtle...


100%|██████████| 21271/21271 [01:40<00:00, 211.91it/s]



✅ Saved full dataset with originals and augmentations to training_data/full_image_identity.csv


In [4]:
# Build [image_path, identity] mapping
base_path = 'training_data'
final_records = []
# Create lookup for image filename → identity
metadata_df['filename'] = metadata_df['path'].apply(lambda p: os.path.basename(p))
filename_to_identity = dict(zip(metadata_df['filename'], metadata_df['identity']))


# Build full [image_path, species] mapping
final_records = []

for species in os.listdir(base_path):
    species_path = os.path.join(base_path, species)
    if not os.path.isdir(species_path):
        continue

    for img_file in os.listdir(species_path):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            identity = filename_to_identity.get(img_file, None)
            if identity is not None:  # only include labeled identities
                img_path = os.path.join(species_path, img_file)
                final_records.append([img_path, identity])

# Save to CSV
df_all = pd.DataFrame(final_records, columns=['image_path', 'identity'])
df_all.to_csv(os.path.join(base_path, 'full_image_identity.csv'), index=False)

print(f"\n✅ Saved full dataset to {os.path.join(base_path, 'full_image_identity.csv')}")



✅ Saved full dataset to training_data/full_image_identity.csv
